In [1]:
import pyspark
import json

sc = spark.sparkContext

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1587579532010_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def load_dictionary(file_path):
    senti_dict = {}
    pos_scores = []
    neu_scores = []
    neg_scores = []
    count = []
    index = 0
    raw_data = sc.textFile(file_path)

    for line in raw_data.collect():
        if line.startswith("#"):
            continue
        cols = line.split("\t")
        try:
            for entry in cols[4].split():
                word = entry.split("#")[0]
                pos_score = float(cols[2])
                neg_score = float(cols[3])
                neu_score = 1 - pos_score - neg_score
                if word not in senti_dict.keys():
                    senti_dict[word] = index
                    count.append(1)
                    pos_scores.append(pos_score)
                    neg_scores.append(neg_score)
                    neu_scores.append(neu_score)
                    index += 1
                else:
                    count[senti_dict[word]] += 1
                    pos_scores[senti_dict[word]] += pos_score
                    neg_scores[senti_dict[word]] += neg_score
                    neu_scores[senti_dict[word]] += neu_score
        except ValueError:
            continue

    for i in senti_dict.values():
        pos_scores[i] /= count[i]
        neg_scores[i] /= count[i]
        neu_scores[i] /= count[i]
        
    return senti_dict, pos_scores, neu_scores, neg_scores
        
senti_dict, pos_scores, neu_scores, neg_scores =  load_dictionary("s3://ml-team15/SentiWordNet_3.0.0.txt")

senti_dict_bc = sc.broadcast(senti_dict)
pos_scores_bc = sc.broadcast(pos_scores)
neu_scores_bc = sc.broadcast(neu_scores)
neg_scores_bc = sc.broadcast(neg_scores)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def encode(sentence, senti_dict):
    sentence_split = sentence.split()
    if len(sentence_split) <= 3 or len(sentence_split) >= 50:
        encoded = []
    else:
        encoded = [senti_dict[word] for word in sentence_split if word in senti_dict]
    return encoded, len(encoded) + 1e-9

def get_scores(sentence, senti_dict, pos_scores, neu_scores, neg_scores):
    pos_score = 0
    neu_score = 0
    neg_score = 0
    encoded, encoded_len = encode(sentence, senti_dict)
    for index in encoded:
        pos_score += pos_scores[index]
        neu_score += neu_scores[index]
        neg_score += neg_scores[index]
    return pos_score / encoded_len, neu_score / encoded_len, neg_score / encoded_len

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def filter_patition(chunks):
    filtered = []
    senti_dict = senti_dict_bc.value
    pos_scores = pos_scores_bc.value
    neu_scores = neu_scores_bc.value
    neg_scores = neg_scores_bc.value
    for chunk in chunks:
        json_data = json.loads(chunk)
        if "created_utc" not in json_data or "subreddit" not in json_data or "body" not in json_data:
            continue
        senti_scores = get_scores(json_data["body"], senti_dict,
                                 pos_scores, neu_scores, neg_scores)
        if senti_scores[0] + senti_scores[1] + senti_scores[2] == 0:
            continue
        filtered.append((json_data["created_utc"], json_data["subreddit"], senti_scores))
    return filtered

num_cores = 16
num_partitions = num_cores * 600

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
data_path = 's3://ml-team15/2015_reddit_comments_corpus/reddit_data/2007/RC_20??-??.bz2'
data = sc.textFile(data_path, minPartitions=num_partitions).mapPartitions(filter_patition)
data.repartition(num_cores * 5).saveAsTextFile("s3://ml-team15/reddit_senti_2007")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
data_path = 's3://ml-team15/2015_reddit_comments_corpus/reddit_data/2008/RC_20??-??.bz2'
data = sc.textFile(data_path, minPartitions=num_partitions).mapPartitions(filter_patition)
data.repartition(num_cores * 5).saveAsTextFile("s3://ml-team15/reddit_senti_2008")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
data_path = 's3://ml-team15/2015_reddit_comments_corpus/reddit_data/2009/RC_20??-??.bz2'
data = sc.textFile(data_path, minPartitions=num_partitions).mapPartitions(filter_patition)
data.repartition(num_cores * 5).saveAsTextFile("s3://ml-team15/reddit_senti_2009")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
data_path = 's3://ml-team15/2015_reddit_comments_corpus/reddit_data/2010/RC_20??-??.bz2'
data = sc.textFile(data_path, minPartitions=num_partitions).mapPartitions(filter_patition)
data.repartition(num_cores * 5).saveAsTextFile("s3://ml-team15/reddit_senti_2010")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
data_path = 's3://ml-team15/2015_reddit_comments_corpus/reddit_data/2011/RC_20??-??.bz2'
data = sc.textFile(data_path, minPartitions=num_partitions).mapPartitions(filter_patition)
data.repartition(num_cores * 5).saveAsTextFile("s3://ml-team15/reddit_senti_2011")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
data_path = 's3://ml-team15/2015_reddit_comments_corpus/reddit_data/2012/RC_20??-??.bz2'
data = sc.textFile(data_path, minPartitions=num_partitions).mapPartitions(filter_patition)
data.repartition(num_cores * 5).saveAsTextFile("s3://ml-team15/reddit_senti_2012")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
data_path = 's3://ml-team15/2015_reddit_comments_corpus/reddit_data/2013/RC_20??-??.bz2'
data = sc.textFile(data_path, minPartitions=num_partitions).mapPartitions(filter_patition)
data.repartition(num_cores * 5).saveAsTextFile("s3://ml-team15/reddit_senti_2013")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
data_path = 's3://ml-team15/2015_reddit_comments_corpus/reddit_data/2014/RC_20??-??.bz2'
data = sc.textFile(data_path, minPartitions=num_partitions).mapPartitions(filter_patition)
data.repartition(num_cores * 20).saveAsTextFile("s3://ml-team15/reddit_senti_2014")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
data_path = 's3://ml-team15/2015_reddit_comments_corpus/reddit_data/2015/RC_20??-??.bz2'
data = sc.textFile(data_path, minPartitions=num_partitions).mapPartitions(filter_patition)
data.repartition(num_cores * 5).saveAsTextFile("s3://ml-team15/reddit_senti_2015")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…